In [1]:
from flask import Flask, abort, request
from flask_basicauth import BasicAuth
import pymysql
import os
import json
import math
from collections import defaultdict

In [18]:
def player(player_slug):
    db_conn = pymysql.connect(
    host="localhost",
    user="root", 
    password=os.getenv('MySQLpwd'), 
    database="final_project",
    cursorclass=pymysql.cursors.DictCursor  
    )
    with db_conn.cursor() as cursor:
        cursor.execute("""
            SELECT 
                player_name,
                age,
                u23_eligible
            FROM players_scores
            WHERE player_slug=%s;
            """, (player_slug,))
        player = cursor.fetchone()
        if not player:
            abort(404)
    with db_conn.cursor() as cursor:
        cursor.execute("""
            SELECT 
                gameweek_date,
                gameweek_number,
                L5,
                L15,
                regular_status,
                Start15
            FROM players_scores
            WHERE player_slug = %s
            ORDER BY gameweek_date;
        """, (player_slug,))
        so5_stats = cursor.fetchall()
        player['so5_stats'] = so5_stats
    db_conn.close()
    return player
    
        

In [19]:
player("jude-bellingham")

{'player_name': 'Jude Bellingham',
 'age': 20,
 'u23_eligible': 1,
 'so5_stats': [{'gameweek_date': datetime.datetime(2019, 8, 25, 11, 0),
   'gameweek_number': 23,
   'L5': 0.0,
   'L15': 0.0,
   'regular_status': 'reserve',
   'Start15': 0.0},
  {'gameweek_date': datetime.datetime(2019, 8, 31, 14, 0),
   'gameweek_number': 24,
   'L5': 15.8,
   'L15': 15.8,
   'regular_status': 'reserve',
   'Start15': 25.0},
  {'gameweek_date': datetime.datetime(2019, 9, 14, 14, 0),
   'gameweek_number': 25,
   'L5': 38.55,
   'L15': 38.55,
   'regular_status': 'reserve',
   'Start15': 60.0},
  {'gameweek_date': datetime.datetime(2019, 9, 21, 14, 0),
   'gameweek_number': 26,
   'L5': 46.5,
   'L15': 46.5,
   'regular_status': 'reserve',
   'Start15': 95.0},
  {'gameweek_date': datetime.datetime(2019, 9, 28, 14, 0),
   'gameweek_number': 27,
   'L5': 43.38,
   'L15': 43.38,
   'regular_status': 'reserve',
   'Start15': 130.0},
  {'gameweek_date': datetime.datetime(2019, 10, 4, 18, 45),
   'gameweek_

In [33]:
def players():
    # URL parameters
    # to input in .py file
    page = 1

    db_conn = pymysql.connect(
        host="localhost",
        user="root",
        password=os.getenv('MySQLpwd'),
        database="final_project",
        cursorclass=pymysql.cursors.DictCursor)
    # Get the players
    with db_conn.cursor() as cursor:
        cursor.execute("""
        SELECT
            player_slug,
            player_name,
            age,
            u23_eligible
        FROM players_scores
        ORDER BY player_slug
        LIMIT %s
        OFFSET %s
        """, (page_size, page * page_size))
        players = cursor.fetchall()
        player_slugs = [player['player_slug'] for player in players]
        placeholders = ', '.join(['%s'] * len(player_slugs))

    if include_details:
        # Get so5_stats
        with db_conn.cursor() as cursor:
            placeholder = ','.join(['%s'] * len(player_slugs))
            cursor.execute(f"""
                SELECT 
                    gameweek_date,
                    gameweek_number,
                    L5,
                    L15,
                    regular_status,
                    Start15,
                    player_slug  # Add player_slug column to the SELECT statement
                FROM players_scores
                WHERE player_slug in ({placeholder})
                ORDER BY gameweek_date;
            """, player_slugs)
            stats = cursor.fetchall()
            stats_dict = defaultdict(list)
            for obj in stats:
                player_slug = obj['player_slug']
                del obj['player_slug']
                stats_dict[player_slug].append(obj)

        # Merge so5_stats into players
        for player in players:
            player_slug = player['player_slug']
            player['stats'] = stats_dict[player_slug]

    # Get the total players count
    with db_conn.cursor() as cursor:
        cursor.execute("SELECT COUNT(*) AS total FROM players")
        total = cursor.fetchone()
        last_page = math.ceil(total['total'] / page_size)

    db_conn.close()
    return {
        'players': players,
        'next_page': f'/players?page={page + 1}&page_size={page_size}&include_details={int(include_details)}',
        'last_page': f'/players?page={last_page}&page_size={page_size}&include_details={int(include_details)}',
    }

In [34]:
players()

{'players': [{'player_slug': 'aaron-escandell-banacloche',
   'player_name': 'Aarón Escandell',
   'age': 28,
   'u23_eligible': 0,
   'stats': [{'gameweek_date': datetime.datetime(2020, 2, 8, 20, 0),
     'gameweek_number': 44,
     'L5': 0.0,
     'L15': 0.0,
     'regular_status': 'reserve',
     'Start15': 0.0},
    {'gameweek_date': datetime.datetime(2020, 6, 28, 17, 30),
     'gameweek_number': 76,
     'L5': 41.9,
     'L15': 41.9,
     'regular_status': 'reserve',
     'Start15': 35.0},
    {'gameweek_date': datetime.datetime(2020, 7, 1, 17, 30),
     'gameweek_number': 77,
     'L5': 42.2,
     'L15': 42.2,
     'regular_status': 'reserve',
     'Start15': 70.0},
    {'gameweek_date': datetime.datetime(2020, 9, 27, 14, 0),
     'gameweek_number': 102,
     'L5': 51.2,
     'L15': 51.2,
     'regular_status': 'reserve',
     'Start15': 105.0},
    {'gameweek_date': datetime.datetime(2020, 10, 1, 17, 0),
     'gameweek_number': 103,
     'L5': 38.4,
     'L15': 38.4,
     'regul

In [35]:
from collections import defaultdict
from flask_swagger_ui import get_swaggerui_blueprint

ModuleNotFoundError: No module named 'flask_swagger_ui'

In [36]:
pip install Flask_swagger_ui


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 21.7 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [37]:
from flask import abort